In [65]:
#import dependencies
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint

#import Socrata to pull API data...may need to run pip install sodapy
from sodapy import Socrata

#import password for mySQL from config.py.
from config import password

#import app_token (if desired) from app_token.py to prevent limited pull requests
from app_token import app_token

In [66]:
#call Socrata without app_token
client = Socrata("data.cdc.gov", None)

# # Example authenticated client (needed for non-public datasets):
# client = Socrata("data.cdc.gov", app_token)

#requests result, limit is 25000 to ensure full data is pulled
results = client.get("a3uk-kgrx", limit="25000")

# Convert to pandas DataFrame
drug_df = pd.DataFrame.from_records(results)

In [74]:
#push dataframe contents to mysql

import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine

dbuser = 'root'
#read in password from config.py file
dbpassword = password
dbhost = 'localhost'
dbport = '3306'

#database name is Project2 (need to create this locally)
dbname= 'Project2'

engine = create_engine(f"mysql://{dbuser}:{dbpassword}@{dbhost}:{dbport}/{dbname}")

# state_cons_df
drug_df.to_sql('drug', engine, index=True, if_exists="replace")